In [10]:
import sys
sys.path.append('../')
from setting import config_read

In [11]:
import rdflib
from mapping import ResourceMapper
from graph import *

In [12]:
config = config_read('../')

rm = ResourceMapper(config)
cgg = ConceptualGraphGenerator(config)
qgg = QueryGraphGenerator(config)

In [13]:
g = rdflib.Graph()
g.parse(config['owl']['path'])

SKMO = rdflib.Namespace("http://www.sktelecom.com/skmo/")
g.bind("skmo", SKMO)
SCHEMA = rdflib.Namespace("https://schema.org/")
g.bind("temp", SCHEMA)
SCHEMA = rdflib.Namespace("http://schema.org/")
g.bind("schema", SCHEMA)

In [132]:
# user_input = '가수 태연의 첫 데뷔곡은?'
# user_input = '뱅뱅뱅 부른 음악그룹의 소속사는?'
# user_input = '용감한 형제가 작곡한 노래의 장르는?'
# user_input = '장르가 발라드인 노래'
# user_input = '버벌진트가 피처링한 노래'
# user_input = '소녀시대의 리더는?'
user_input = '제니의 소속 그룹'
resource_combination = rm.process(user_input)
conceptual_graph = cgg.generate_all_conceptual_graph(resource_combination)
query_graph = qgg.generate_query_graph(conceptual_graph)

In [133]:
final_query_graph = sorted(query_graph, key=lambda x: (len(x[1]), -x[0]))[0][1]
final_query_graph

[('foaf:Person(skmo:제니)', 'mo:member_of', 'mo:MusicGroup'),
 ('mo:MusicGroup', 'rdfs:subClassOf', 'foaf:Group')]

In [134]:
relation_triples = []

for s,p,o in final_query_graph:
    if p == 'rdfs:subClassOf': continue
    if (s,p,o) in relation_triples: continue
    
    relation_triples.append((s,p,o))

In [135]:
knows_query = \
f"""
SELECT DISTINCT ?target
WHERE
"""
prev_s = ''
if relation_triples[-1][-1][-1] == ')':
    target = relation_triples[-1][0]
else:
    target = relation_triples[-1][-1]
var_dict = {target:'?target'}
c = itertools.count()

knows_query += '{'


for s,p,o in relation_triples:
    if s[-1] == ')' and o[-1] == ')':
        continue

    # domain이 instance 일때
    if s[-1] == ')':
        s_var = s[:-1].split('(')[-1]
    # domain이 class 일때
    else:
        if s in var_dict:
            s_var = var_dict[s]
        else:
            s_var = f'?x{next(c)}'
            var_dict[s] = s_var

    # range가 instance 일때
    if o[-1] == ')':
        o_var = o[:-1].split('(')[-1]
    # range가 class 일때
    else:
        if o in var_dict:
            o_var = var_dict[o]
        else:
            o_var = f'?x{next(c)}'
            var_dict[o] = o_var

    if prev_s == s_var:
        s_var = ';'
    else:
        if not prev_s == '':
            knows_query += '.'
        prev_s = s_var
        knows_query += '\n    '
    
    knows_query += (' ').join([s_var, p, o_var])
    
knows_query += '\n}'
print(knows_query)

qres = g.query(knows_query)

print()

for row in qres:
    print(row['target'])


SELECT DISTINCT ?target
WHERE
{
    skmo:제니 mo:member_of ?target
}

http://www.sktelecom.com/skmo/블랙핑크
